# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [31]:
# Dependencies and Setup
#import hvplot.pandas

import pandas as pd
import requests
import gmaps



# Import API key
from api_keys3 import goo_key
from api_keys3 import geoapify_key

In [32]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,severo-kurilsk,50.6789,156.1250,37.96,94,100,7.92,RU,1667714226
1,1,mahajanga,-15.7167,46.3167,82.42,61,20,9.22,MG,1667714227
2,2,cockburn town,21.4612,-71.1419,79.72,73,67,19.62,TC,1667714227
3,3,izumo,35.3667,132.7667,64.35,51,20,5.75,JP,1667714227
4,4,mahebourg,-20.4081,57.7000,80.26,69,40,11.50,MU,1667714228
...,...,...,...,...,...,...,...,...,...,...
556,556,high rock,26.6208,-78.2833,79.21,74,0,19.28,BS,1667714519
557,557,nome,64.5011,-165.4064,28.11,83,100,15.46,US,1667714520
558,558,sawtell,-30.3833,153.1000,71.19,68,22,10.36,AU,1667714520
559,559,the valley,18.2170,-63.0578,79.11,89,100,23.87,AI,1667714521


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [33]:
%%capture --no-display

gmaps.configure(api_key=goo_key)

# Configure the map plot
locations = city_data_df[["Lat","Lng"]].astype(float)
humidity = city_data_df["Humidity"].astype(float)

# Display the map
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=12, point_radius=1)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [42]:
# Narrow down cities that fit criteria and drop any results with null values
city_data = city_data_df.loc[(city_data_df['Max Temp'] >= 68) & (city_data_df['Max Temp'] < 100)]
city_data = city_data_df.loc[(city_data_df['Wind Speed'] < 20)]
city_data = city_data_df.loc[(city_data_df['Humidity'] < 85)]
# Drop any rows with null values
city_data.dropna(inplace = True)

# Display sample data
city_data

c:\Users\basse\anaconda3\envs\pythondata\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,mahajanga,-15.7167,46.3167,82.42,61,20,9.22,MG,1667714227
2,2,cockburn town,21.4612,-71.1419,79.72,73,67,19.62,TC,1667714227
3,3,izumo,35.3667,132.7667,64.35,51,20,5.75,JP,1667714227
4,4,mahebourg,-20.4081,57.7000,80.26,69,40,11.50,MU,1667714228
8,8,souillac,-20.5167,57.5167,79.18,69,40,11.50,MU,1667714229
...,...,...,...,...,...,...,...,...,...,...
553,553,serov,59.6033,60.5787,26.15,77,88,4.92,RU,1667714517
555,555,fairbanks,64.8378,-147.7164,3.92,83,21,2.13,US,1667714519
556,556,high rock,26.6208,-78.2833,79.21,74,0,19.28,BS,1667714519
557,557,nome,64.5011,-165.4064,28.11,83,100,15.46,US,1667714520


### Step 3: Create a new DataFrame called `hotel_df`.

In [43]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df = hotel_df.reindex(columns = hotel_df.columns.tolist() + ['Hotel Name'])

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
1,mahajanga,MG,-15.7167,46.3167,61,NaN
2,cockburn town,TC,21.4612,-71.1419,73,NaN
3,izumo,JP,35.3667,132.7667,51,NaN
4,mahebourg,MU,-20.4081,57.7000,69,NaN
8,souillac,MU,-20.5167,57.5167,69,NaN
...,...,...,...,...,...,...
553,serov,RU,59.6033,60.5787,77,NaN
555,fairbanks,US,64.8378,-147.7164,83,NaN
556,high rock,BS,26.6208,-78.2833,74,NaN
557,nome,US,64.5011,-165.4064,83,NaN


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [50]:
# Set parameters to search for a hotel
radius = 20000
params = {
    "categories":"accommodation",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lng},{lat},{radius}'
    params["bias"] = f'proximity:{lng},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
mahajanga - nearest hotel: Vahiny Hotel
cockburn town - nearest hotel: Manta House
izumo - nearest hotel: ホテル武志山荘
mahebourg - nearest hotel: Le Bamboo
souillac - nearest hotel: Le Rochester
ribeira grande - nearest hotel: Quinta das Bunganvílias
rikitea - nearest hotel: Pension Kura Inn
port lincoln - nearest hotel: YHA Port Lincoln
mataura - nearest hotel: Charlton Motel
nara - nearest hotel: JWマリオットホテル奈良
busselton - nearest hotel: Gale Street Motel & Villas
san isidro - nearest hotel: Casa de Baraybar
kapaa - nearest hotel: Pono Kai Resort
menongue - nearest hotel: No hotel found
hermanus - nearest hotel: Aloe House
cape town - nearest hotel: Townhouse Hotel
alofi - nearest hotel: Stone Villas
east london - nearest hotel: No hotel found
calvinia - nearest hotel: Hantam Huis
miles city - nearest hotel: Historic Olive Hotel
bandarbeyla - nearest hotel: No hotel found
dalbandin - nearest hotel: SF Al-dawood Restaurant
shahreza - nearest hotel: هتل و رستوران منتظران

,City,Country,Lat,Lng,Humidity,Hotel Name
1,mahajanga,MG,-15.7167,46.3167,61,Vahiny Hotel
2,cockburn town,TC,21.4612,-71.1419,73,Manta House
3,izumo,JP,35.3667,132.7667,51,ホテル武志山荘
4,mahebourg,MU,-20.4081,57.7000,69,Le Bamboo
8,souillac,MU,-20.5167,57.5167,69,Le Rochester
...,...,...,...,...,...,...
553,serov,RU,59.6033,60.5787,77,отель Надеждинский
555,fairbanks,US,64.8378,-147.7164,83,Westmark Fairbanks Hotel
556,high rock,BS,26.6208,-78.2833,74,No hotel found
557,nome,US,64.5011,-165.4064,83,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [53]:
%%capture --no-display

# Configure the map plot
hotels = hotel_df["Hotel Name"].tolist()

coords = hotel_df[["Lat", "Lng"]]

# Display the map
fig = gmaps.figure()
markers = gmaps.marker_layer(coords,
    info_box_content=[f"Nearby Hotel: {hotel}" for hotel in hotels])
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))